In [375]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression

pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [346]:
cd /Users/pwalesdi/Desktop/GA/NBA_Player_Prediction/

/Users/pwalesdi/Desktop/GA/NBA_Player_Prediction


In [347]:
%store -r nba
%store -r improvement
%store -r per_improvement

In [348]:
# Creating DFs for second, third .... year players
rookie_year = nba.loc[(nba['SEASON'] ==  nba['DRAFT_YEAR+1'])]
rookie_year = rookie_year.loc[rookie_year["MP"] > 1200]
second_year = nba.loc[(nba['SEASON'] ==  nba['DRAFT_YEAR+1'] + 1)]
second_year = second_year.loc[second_year["MP"] > 1200]


In [268]:
rookie_year.loc[(rookie_year["PER"] > 20) & (rookie_year["MP"] > 1000)]

,Player_name,player_id,SEASON,Tm_x,DRAFT_YEAR+1,Draft_team,Pk,Pos,Age,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,College,Yrs,PTS,TRB,AST,FG%,3P%,FT%,PPG,RPG,APG,draft_round,target
28,Deandre Ayton,aytonde01,2019,PHO,2019.0,PHO,1.0,C,20,71,2183,20.5,0.608,0.005,0.217,11.1,26.1,18.5,9.6,1.3,2.6,11.7,21.2,4.1,1.7,5.8,0.128,-0.1,0.2,0.2,1.2,University of Arizona,1.0,1159.0,729.0,125.0,0.585,0.000,0.746,16.3,10.3,1.8,1.0,0
567,Mitchell Robinson,robinmi01,2019,NYK,2019.0,NYK,36.0,C,20,66,1360,22.0,0.692,0.000,0.464,13.3,19.4,16.3,4.2,1.8,10.0,9.1,12.1,4.0,2.2,6.1,0.217,0.5,5.2,5.7,2.7,NaN,1.0,485.0,423.0,37.0,0.694,NaN,0.600,7.3,6.4,0.6,2.0,0
2163,Karl-Anthony Towns,townska01,2016,MIN,2016.0,MIN,1.0,C,20,82,2627,22.5,0.590,0.076,0.239,10.3,27.5,19.0,11.3,1.1,4.3,12.6,24.9,5.5,2.8,8.3,0.151,1.6,1.1,2.8,3.2,University of Kentucky,4.0,7187.0,3831.0,839.0,0.537,0.392,0.836,22.3,11.9,2.6,1.0,0
3283,Anthony Davis,davisan02,2013,NOP,2013.0,NOH,1.0,PF,19,64,1846,21.7,0.559,0.009,0.333,10.5,23.5,16.8,6.1,2.2,5.1,10.3,21.8,3.7,2.4,6.1,0.159,0.5,1.9,2.3,2.0,University of Kentucky,7.0,11059.0,4906.0,982.0,0.517,0.314,0.795,23.7,10.5,2.1,1.0,0
3298,Andre Drummond,drumman01,2013,DET,2013.0,DET,9.0,C,19,60,1243,21.6,0.578,0.006,0.465,15.4,27.2,21.2,4.1,2.5,6.1,12.2,17.2,2.2,2.3,4.5,0.172,-0.3,2.6,2.4,1.4,University of Connecticut,7.0,7661.0,7424.0,626.0,0.543,0.147,0.448,14.1,13.7,1.2,1.0,0
3879,Kyrie Irving,irvinky01,2012,CLE,2012.0,CLE,1.0,PG,19,51,1558,21.4,0.566,0.245,0.262,3.1,11.2,7.0,36.5,1.8,1.0,16.1,28.7,3.4,0.6,4.1,0.125,4.9,-1.7,3.3,2.1,Duke University,8.0,11294.0,1852.0,2884.0,0.465,0.390,0.875,22.2,3.6,5.7,1.0,0


In [269]:
second_year.loc[(second_year["PER"] > 20) & (second_year["MP"] > 1000)]

,Player_name,player_id,SEASON,Tm_x,DRAFT_YEAR+1,Draft_team,Pk,Pos,Age,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,College,Yrs,PTS,TRB,AST,FG%,3P%,FT%,PPG,RPG,APG,draft_round,target
90,Thomas Bryant,bryanth01,2019,WAS,2018.0,UTA,42.0,C,21,72,1496,21.0,0.674,0.197,0.273,8.1,25.1,16.5,9.3,0.8,3.8,9.6,17.6,4.3,1.3,5.6,0.178,1.2,0.4,1.6,1.3,Indiana University,2.0,780.0,471.0,98.0,0.606,0.312,0.767,9.0,5.4,1.1,2.0,0
139,John Collins,collijo01,2019,ATL,2018.0,ATL,19.0,PF,21,61,1829,21.8,0.627,0.190,0.320,12.3,22.1,17.1,11.0,0.6,1.8,11.2,23.7,5.0,1.0,6.0,0.158,2.6,-1.2,1.4,1.6,Wake Forest University,2.0,1965.0,1136.0,219.0,0.566,0.346,0.743,14.6,8.4,1.6,1.0,0
1680,Karl-Anthony Towns,townska01,2017,MIN,2016.0,MIN,1.0,C,21,82,3030,25.9,0.618,0.186,0.289,11.2,27.8,19.4,13.2,0.9,2.9,11.3,27.5,9.9,2.8,12.7,0.201,4.7,0.2,4.9,5.3,University of Kentucky,4.0,7187.0,3831.0,839.0,0.537,0.392,0.836,22.3,11.9,2.6,1.0,0
1969,Nikola Jokić,jokicni01,2016,DEN,2015.0,DEN,41.0,C,20,80,1733,21.5,0.582,0.140,0.317,11.4,24.7,17.9,18.1,2.3,2.3,13.2,19.9,4.5,2.2,6.7,0.185,2.6,2.2,4.8,3.0,NaN,4.0,5006.0,2946.0,1586.0,0.523,0.345,0.828,16.3,9.6,5.1,2.0,0
2382,Rudy Gobert,goberru01,2015,UTA,2014.0,DEN,27.0,C,22,82,2158,21.6,0.627,0.005,0.639,14.3,27.2,20.7,8.2,1.6,7.0,16.9,14.0,5.0,4.3,9.3,0.206,0.7,5.1,5.8,4.3,NaN,6.0,4524.0,4275.0,545.0,0.629,0.000,0.631,11.1,10.5,1.3,1.0,0
2817,Anthony Davis,davisan02,2014,NOP,2013.0,NOH,1.0,PF,20,67,2358,26.5,0.582,0.009,0.438,10.0,23.3,16.6,8.0,2.0,6.7,8.3,25.2,7.2,3.2,10.4,0.212,2.3,2.2,4.5,3.9,University of Kentucky,7.0,11059.0,4906.0,982.0,0.517,0.314,0.795,23.7,10.5,2.1,1.0,0
2831,Andre Drummond,drumman01,2014,DET,2013.0,DET,9.0,C,20,81,2619,22.6,0.599,0.003,0.427,17.5,27.7,22.3,2.2,2.0,4.0,10.7,16.7,6.4,3.5,9.9,0.182,1.1,0.5,1.6,2.4,University of Connecticut,7.0,7661.0,7424.0,626.0,0.543,0.147,0.448,14.1,13.7,1.2,1.0,0
3388,Kyrie Irving,irvinky01,2013,CLE,2012.0,CLE,1.0,PG,20,59,2048,21.4,0.553,0.261,0.271,1.8,10.8,6.1,32.7,2.3,0.8,13.8,30.2,4.2,1.1,5.3,0.125,4.9,-1.6,3.3,2.8,Duke University,8.0,11294.0,1852.0,2884.0,0.465,0.390,0.875,22.2,3.6,5.7,1.0,0
3749,DeMarcus Cousins,couside01,2012,SAC,2011.0,SAC,5.0,C,21,64,1950,21.7,0.499,0.014,0.373,14.2,25.9,19.8,9.5,2.4,2.8,12.8,29.7,2.1,2.2,4.2,0.104,-0.4,0.0,-0.3,0.8,University of Kentucky,9.0,12006.0,6131.0,1832.0,0.461,0.332,0.737,21.2,10.9,3.2,1.0,0
3966,Greg Monroe,monrogr01,2012,DET,2011.0,DET,7.0,C,21,66,2082,22.0,0.563,0.001,0.348,13.3,23.6,18.3,14.0,2.1,1.7,15.2,23.6,4.3,2.7,7.0,0.161,2.4,1.4,3.8,3.1,Georgetown University,9.0,8326.0,5229.0,1354.0,0.514,0.059,0.704,13.2,8.3,2.1,1.0,0


In [351]:
# Remove is all of the players that had a PER  > 20 in their rookie or second year. 
# They don't count in this evaluation, so they are removed
remove = second_year.loc[(second_year["PER"] > 20) & (second_year["MP"] > 1000)]
names = remove.player_id.unique()
for name in names:
    nba = nba.loc[nba["player_id"] != name]

# assigning a 1 to the target column for any player who met the below criteria in their 3rd - 7th season
nba["target"] = 0
for i in range(2, 6):
    nba.loc[(nba["PER"] > 20) & (nba["MP"] > 1000) & (nba['SEASON'] ==  nba['DRAFT_YEAR+1'] + i), "target"] = 1    

In [271]:
len(nba)

6344

In [284]:
nba.sort_values(by="target", ascending=False)
nba.groupby("target").describe().T

target                         0             1
SEASON       count   6262.000000     82.000000
             mean    2012.437240   2015.341463
             std        4.235347      2.520139
             min     2006.000000   2009.000000
             25%     2009.000000   2013.250000
             50%     2012.000000   2016.000000
             75%     2016.000000   2017.000000
             max     2019.000000   2019.000000
DRAFT_YEAR+1 count   2657.000000     82.000000
             mean    2011.633798   2011.609756
             std        3.377296      2.512959
             min     2007.000000   2007.000000
             25%     2009.000000   2010.000000
             50%     2011.000000   2012.000000
             75%     2014.000000   2013.000000
             max     2019.000000   2017.000000
Pk           count   2657.000000     82.000000
             mean      23.540836     15.914634
             std       14.936409     15.109641
             min        1.000000      1.000000
             25%       11.000000      4.000000
             50%       22.000000     10.000000
             75%       35.000000     25.000000
             max       60.000000     60.000000
Age          count   6262.000000     82.000000
             mean      26.564516     24.439024
             std        4.283020      1.532347
             min       18.000000     22.000000
             25%       23.000000     23.000000
             50%       26.000000     24.000000
             75%       30.000000     25.750000
             max       44.000000     28.000000
G            count   6262.000000     82.000000
             mean      50.165442     70.707317
             std       25.927351      9.929987
             min        1.000000     39.000000
             25%       27.000000     64.250000
             50%       57.000000     74.000000
             75%       74.000000     78.000000
             max       82.000000     82.000000
MP           count   6262.000000     82.000000
             mean    1165.986745   2221.256098
             std      881.803652    559.296821
             min        0.000000   1121.000000
             25%      341.000000   1856.500000
             50%     1055.000000   2264.500000
             75%     1878.000000   2682.000000
             max     3424.000000   3211.000000
PER          count   6257.000000     82.000000
             mean      12.533482     22.579268
             std        6.607705      2.275296
             min      -54.400000     20.100000
             25%        9.700000     20.800000
             50%       12.700000     21.550000
             75%       15.800000     23.875000
             max      133.800000     30.900000
TS%          count   6227.000000     82.000000
             mean       0.515725      0.589159
             std        0.098669      0.035713
             min        0.000000      0.521000
             25%        0.486000      0.560500
             50%        0.528000      0.587500
             75%        0.563000      0.610000
             max        1.500000      0.682000
3PAr         count   6227.000000     82.000000
             mean       0.257232      0.183390
             std        0.220809      0.168952
             min        0.000000      0.000000
             25%        0.019000      0.007500
             50%        0.251000      0.187000
             75%        0.420000      0.331500
             max        1.000000      0.557000
FTr          count   6227.000000     82.000000
             mean       0.291693      0.376829
             std        0.241335      0.099296
             min        0.000000      0.143000
             25%        0.171000      0.313250
             50%        0.253000      0.373000
             75%        0.358000      0.441750
             max        6.000000      0.592000
ORB%         count   6257.000000     82.000000
             mean       5.445965      7.173171
             std        5.183784      4.803164
             min        0.000000      1.200000
     

In [286]:
nba.sort_values(by=["target", "Player_name"], ascending=False)

,Player_name,player_id,SEASON,Tm_x,DRAFT_YEAR+1,Draft_team,Pk,Pos,Age,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,College,Yrs,PTS,TRB,AST,FG%,3P%,FT%,PPG,RPG,APG,draft_round,target
1089,Victor Oladipo,oladivi01,2018,IND,2014.0,ORL,2.0,SG,25,75,2552,23.1,0.577,0.323,0.274,2.1,15.1,8.6,21.2,3.5,2.0,12.7,30.1,4.3,4.0,8.2,0.155,3.6,1.3,4.9,4.5,Indiana University,6.0,7028.0,1859.0,1589.0,0.443,0.352,0.793,17.5,4.6,4.0,1.0,1
4053,Tiago Splitter,splitti01,2012,SAS,2008.0,SAS,28.0,C,27,59,1121,20.5,0.649,0.000,0.528,9.7,21.4,15.7,9.2,1.0,3.0,17.1,20.7,3.0,1.6,4.6,0.197,1.1,1.4,2.5,1.3,NaN,7.0,2816.0,1779.0,423.0,0.555,0.143,0.697,7.9,5.0,1.2,1.0,1
710,Steven Adams,adamsst01,2018,OKC,2014.0,OKC,12.0,C,24,76,2487,20.6,0.630,0.003,0.402,16.6,13.9,15.3,5.5,1.8,2.8,13.3,16.7,6.7,3.0,9.7,0.187,2.2,1.1,3.3,3.3,University of Pittsburgh,6.0,4507.0,3446.0,469.0,0.588,0.000,0.553,9.7,7.4,1.0,1.0,1
2321,Stephen Curry,curryst01,2015,GSW,2010.0,GSW,7.0,PG,26,80,2613,28.0,0.638,0.482,0.251,2.4,11.4,7.0,38.6,3.0,0.5,14.3,28.9,11.5,4.1,15.7,0.288,9.6,0.3,9.9,7.9,Davidson College,10.0,16315.0,3132.0,4588.0,0.477,0.436,0.905,23.5,4.5,6.6,1.0,1
2812,Stephen Curry,curryst01,2014,GSW,2010.0,GSW,7.0,PG,25,78,2846,24.1,0.610,0.445,0.252,1.8,10.9,6.4,39.9,2.2,0.4,16.1,28.3,9.3,4.0,13.4,0.225,7.4,0.0,7.4,6.7,Davidson College,10.0,16315.0,3132.0,4588.0,0.477,0.436,0.905,23.5,4.5,6.6,1.0,1
3280,Stephen Curry,curryst01,2013,GSW,2010.0,GSW,7.0,PG,24,78,2983,21.3,0.589,0.432,0.210,2.3,9.1,5.8,31.1,2.1,0.3,13.7,26.4,8.4,2.8,11.2,0.180,6.3,-0.9,5.4,5.6,Davidson College,10.0,16315.0,3132.0,4588.0,0.477,0.436,0.905,23.5,4.5,6.6,1.0,1
3668,Ryan Anderson,anderry01,2012,ORL,2009.0,NJN,21.0,PF,23,61,1964,21.2,0.589,0.557,0.226,13.0,14.7,13.8,5.1,1.4,1.0,6.4,21.2,7.0,1.9,8.9,0.219,5.9,-1.3,4.6,3.3,University of California,11.0,7996.0,3406.0,584.0,0.422,0.380,0.854,12.4,5.3,0.9,1.0,1
3162,Russell Westbrook,westbru01,2014,OKC,2009.0,SEA,4.0,PG,25,46,1412,24.7,0.545,0.271,0.370,4.6,15.9,10.5,40.2,3.1,0.4,16.1,34.4,3.0,2.3,5.2,0.178,5.2,1.2,6.4,3.0,University of California Los Angeles,11.0,18859.0,5760.0,6897.0,0.434,0.308,0.801,23.0,7.0,8.4,1.0,1
3631,Russell Westbrook,westbru01,2013,OKC,2009.0,SEA,4.0,PG,24,82,2861,23.9,0.532,0.195,0.375,4.8,11.8,8.6,38.4,2.6,0.6,13.2,32.8,7.7,3.9,11.6,0.195,4.7,0.3,5.0,5.1,University of California Los Angeles,11.0,18859.0,5760.0,6897.0,0.434,0.308,0.801,23.0,7.0,8.4,1.0,1
4107,Russell Westbrook,westbru01,2012,OKC,2009.0,SEA,4.0,PG,23,66,2331,22.9,0.538,0.155,0.326,5.0,9.4,7.4,29.8,2.5,0.7,14.2,32.7,5.5,2.4,7.9,0.163,3.7,-0.5,3.2,3.0,University of California Los Angeles,11.0,18859.0,5760.0,6897.0,0.434,0.308,0.801,23.0,7.0,8.4,1.0,1


In [297]:
# removing players drafted before 2006
delete = nba.loc[(nba['DRAFT_YEAR+1'].isnull()) & (nba["SEASON"] == 2006)]["player_id"]
nba = nba.loc[~nba["player_id"].isin(delete)]

In [312]:
# Getting the subset of data where target = 1 for later
explore = nba.loc[nba["target"] == 1]

In [364]:
[second_year_prime.loc[second_year_prime["player_id"].isin(explore['player_id'])]

,Player_name,player_id,SEASON,Tm_x,DRAFT_YEAR+1,Draft_team,Pk,Pos,Age,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,College,Yrs,PTS,TRB,AST,FG%,3P%,FT%,PPG,RPG,APG,draft_round,target
1145,Domantas Sabonis,sabondo01,2018,IND,2017.0,ORL,11.0,C,21,74,1810,17.5,0.567,0.056,0.339,10.4,25.2,17.8,12.7,1.1,1.6,15.3,22.1,2.5,2.2,4.7,0.126,-0.8,0.5,-0.3,0.8,Gonzaga University,3.0,2383.0,1550.0,445.0,0.513,0.343,0.722,10.4,6.8,1.9,1.0,0
1300,Devin Booker,bookede01,2017,PHO,2016.0,PHO,13.0,SG,20,78,2730,14.6,0.531,0.283,0.308,1.8,8.3,5.0,16.3,1.3,0.6,12.9,28.6,1.3,0.6,2.0,0.035,0.4,-2.7,-2.3,-0.2,University of Kentucky,4.0,5820.0,945.0,1154.0,0.437,0.354,0.854,21.4,3.5,4.2,1.0,0
1422,Montrezl Harrell,harremo01,2017,HOU,2016.0,HOU,32.0,C,23,58,1064,19.2,0.662,0.020,0.351,8.3,14.2,11.2,9.6,0.9,3.2,9.9,17.6,3.1,1.0,4.1,0.183,0.8,0.2,0.9,0.8,University of Louisville,4.0,2864.0,1125.0,317.0,0.629,0.156,0.629,11.2,4.4,1.2,2.0,0
1448,Richaun Holmes,holmeri01,2017,PHI,2016.0,PHI,37.0,C,23,57,1193,18.6,0.611,0.187,0.250,8.7,20.1,14.4,8.3,1.7,3.7,10.7,18.6,2.2,1.6,3.8,0.151,0.2,1.5,1.7,1.1,Bowling Green State University,4.0,1729.0,988.0,212.0,0.565,0.257,0.706,7.7,4.4,0.9,2.0,0
1610,Kristaps Porzingis,porzikr01,2017,NYK,2016.0,NYK,4.0,PF,21,66,2164,17.4,0.546,0.319,0.256,5.5,18.2,11.8,7.3,1.1,4.9,9.7,24.3,2.4,2.1,4.5,0.100,0.2,0.0,0.2,1.2,NaN,3.0,3312.0,1317.0,248.0,0.437,0.361,0.804,17.8,7.1,1.3,1.0,0
1811,Clint Capela,capelca01,2016,HOU,2015.0,HOU,25.0,C,21,77,1471,18.3,0.553,0.003,0.531,14.4,22.7,18.5,5.1,2.0,5.2,10.9,16.1,2.4,2.0,4.4,0.144,-1.2,1.7,0.5,0.9,NaN,5.0,3532.0,2706.0,276.0,0.635,0.000,0.525,12.0,9.2,0.9,1.0,0
2066,Jusuf Nurkić,nurkiju01,2016,DEN,2015.0,CHI,16.0,C,21,32,547,16.0,0.460,0.012,0.409,12.9,22.7,17.7,12.3,2.2,6.5,16.1,27.2,-0.5,0.9,0.4,0.031,-4.1,2.8,-1.3,0.1,NaN,5.0,3608.0,2480.0,589.0,0.491,0.071,0.666,11.6,8.0,1.9,1.0,0
2094,Dwight Powell,poweldw01,2016,DAL,2015.0,CHH,45.0,PF,24,69,992,16.8,0.557,0.054,0.480,9.3,20.9,15.1,7.2,1.8,2.0,10.0,18.4,1.8,1.3,3.1,0.151,-1.3,1.0,-0.3,0.4,Stanford University,5.0,2491.0,1484.0,305.0,0.553,0.298,0.750,7.5,4.5,0.9,2.0,0
2100,Julius Randle,randlju01,2016,LAL,2015.0,LAL,7.0,PF,21,81,2286,13.9,0.482,0.043,0.323,7.8,32.0,19.5,11.0,1.2,1.0,13.5,21.2,-0.3,1.9,1.6,0.035,-3.7,0.0,-3.6,-0.9,University of Kentucky,5.0,4784.0,2753.0,847.0,0.502,0.307,0.722,15.4,8.9,2.7,1.0,0
2212,Steven Adams,adamsst01,2015,OKC,2014.0,OKC,12.0,C,21,70,1771,14.1,0.549,0.005,0.514,12.2,19.3,15.8,5.5,1.1,3.8,16.8,14.3,1.9,2.2,4.1,0.111,-1.4,1.8,0.4,1.1,University of Pittsburgh,6.0,4507.0,3446.0,469.0,0.588,0.000,0.553,9.7,7.4,1.0,1.0,0


In [367]:
second_year_prime.loc[second_year_prime["player_id"].isin(explore['player_id']), "target"] = 1

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [374]:
second_year_prime.head()

,Player_name,player_id,SEASON,Tm_x,DRAFT_YEAR+1,Draft_team,Pk,Pos,Age,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,College,Yrs,PTS,TRB,AST,FG%,3P%,FT%,PPG,RPG,APG,draft_round,target
4,Bam Adebayo,adebaba01,2019,MIA,2018.0,MIA,14.0,C,21,82,1913,17.9,0.623,0.031,0.465,9.2,24.0,16.6,14.2,1.8,3.0,17.1,15.8,3.4,3.4,6.8,0.171,-0.6,3.6,3.0,2.4,University of Kentucky,2.0,1206.0,978.0,285.0,0.550,0.136,0.728,8.0,6.5,1.9,1.0,0
10,Jarrett Allen,allenja01,2019,BRK,2018.0,BRK,22.0,C,20,80,2096,18.5,0.632,0.079,0.489,9.6,24.0,16.8,7.9,1.0,4.5,13.0,15.9,4.4,3.3,7.6,0.175,-0.7,3.2,2.5,2.4,University of Texas at Austin,2.0,1460.0,1060.0,159.0,0.590,0.183,0.732,9.6,7.0,1.0,1.0,0
11,Kadeem Allen,allenka01,2019,NYK,2018.0,BOS,53.0,SG,26,19,416,16.2,0.574,0.255,0.383,2.0,11.1,6.4,28.7,1.9,0.8,13.6,19.6,0.7,0.2,1.0,0.111,0.5,-1.6,-1.1,0.1,University of Arizona,2.0,208.0,62.0,88.0,0.436,0.362,0.778,5.6,1.7,2.4,2.0,0
18,Ike Anigbogu,anigbik01,2019,IND,2018.0,IND,47.0,C,20,3,6,-6.0,0.000,0.000,0.000,18.9,36.2,27.7,19.4,0.0,14.8,25.0,29.1,-0.1,0.0,-0.1,-0.480,-22.5,8.5,-14.0,0.0,University of California Los Angeles,2.0,13.0,12.0,1.0,0.333,NaN,0.833,0.9,0.9,0.1,2.0,0
22,OG Anunoby,anunoog01,2019,TOR,2018.0,TOR,23.0,SF,21,67,1352,9.8,0.544,0.500,0.153,4.8,10.8,7.9,4.7,1.6,1.3,11.3,15.5,0.3,1.7,2.0,0.070,-1.3,0.1,-1.2,0.3,Indiana University,2.0,907.0,381.0,102.0,0.461,0.351,0.605,6.4,2.7,0.7,1.0,0


In [369]:
y = second_year_prime['target']

In [ ]:
# nba = nba.loc[nba["MP"] > 500]
# half_1 = half_1.loc[half_1["MP"] > 500]
# half_2 = half_2.loc[half_2["MP"] > 500]

In [ ]:
# bins = [0, 10, 20, 30, 40, 50, 60]
# bins_2 = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
# improvement['binned_picks'] = pd.cut(improvement['Pk'], bins)
# improvement['binned_picks_2'] = pd.cut(improvement['Pk'], bins_2)
# per_improvement['binned_picks'] = pd.cut(per_improvement['Pk'], bins)
# per_improvement['binned_picks_2'] = pd.cut(per_improvement['Pk'], bins_2)

In [ ]:
third_year = nba.loc[(nba['SEASON'] ==  nba['DRAFT_YEAR+1'] + 2)]
fourth_year = nba.loc[(nba['SEASON'] ==  nba['DRAFT_YEAR+1'] + 3)]
fifth_year = nba.loc[(nba['SEASON'] ==  nba['DRAFT_YEAR+1'] + 4)]
sixth_year = nba.loc[(nba['SEASON'] ==  nba['DRAFT_YEAR+1'] + 5)]
seventh_year = nba.loc[(nba['SEASON'] ==  nba['DRAFT_YEAR+1'] + 6)]
eight_year = nba.loc[(nba['SEASON'] ==  nba['DRAFT_YEAR+1'] + 7)]
ninth_year = nba.loc[(nba['SEASON'] ==  nba['DRAFT_YEAR+1'] + 8)]
tenth_year = nba.loc[(nba['SEASON'] ==  nba['DRAFT_YEAR+1'] + 9)]
eleventh_year = nba.loc[(nba['SEASON'] ==  nba['DRAFT_YEAR+1'] + 10)]
twelth_year = nba.loc[(nba['SEASON'] ==  nba['DRAFT_YEAR+1'] + 11)]
thirteen_year = nba.loc[(nba['SEASON'] ==  nba['DRAFT_YEAR+1'] + 12)]
eastern = nba.loc[nba['Tm_x'].isin(["MIA", "BRK", "MIL", "TOR", "CHI", "WAS", "ORL", "CHO", "BOS", "ATL", 
                          "IND", "PHI", "DET", "NYK", "CLE"])]
western = nba.loc[nba['Tm_x'].isin(["OKC", "PHO", "DEN", "SAS", "UTA", "POR", "MEM", "DAL", "HOU", "SAC", 
                                "LAL", "MIN", "GSW", "NOP", "LAC"])]